In [1]:
# remotes::install_github('KChen-lab/bindSC')

In [2]:
library(bindSC)

Loading required package: umap

Loading required package: irlba

Loading required package: Matrix

Loading required package: progress

Loading required package: ggplot2

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: DescTools

Loading required package: ggpubr

Loading required package: gtools

Loading required package: reshape2

Loading required package: ggrepel

Loading required package: rdist



In [3]:
setwd("../../../DATA/Mouse_embryo")

In [4]:
genescore <- readRDS("GeneScore.rds")
rna <- readRDS("rna.rds")
library(zellkonverter)
peak <- readH5AD("atac-pp.h5ad")

Registered S3 method overwritten by 'zellkonverter':
  method                                             from      
  py_to_r.pandas.core.arrays.categorical.Categorical reticulate



In [5]:
library(Seurat)
# DefaultAssay(data) <- "Spatial"
# data <- NormalizeData(data)
# data <- ScaleData(data, features = rownames(data))
rna <- NormalizeData(rna, verbose=FALSE)
rna <- FindVariableFeatures(rna, verbose=FALSE, nfeatures=3000)
rna <- ScaleData(rna, verbose=FALSE)
rna <- RunPCA(rna, npcs = 30, verbose = FALSE)
rna <- FindNeighbors(rna, reduction = "pca", dims = 1:30)
rna <- FindClusters(rna)
print(length(unique((rna$seurat_clusters))))

Loading required package: SeuratObject

Loading required package: sp




Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 2187
Number of edges: 95180

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.7894
Number of communities: 13
Elapsed time: 0 seconds
[1] 13


In [6]:
X <- rna@assays$RNA$counts
Y <- peak@assays@data$X

In [7]:
Z0 <- genescore@assays$RNA$counts

In [8]:
# rownames(Z0) <- rownames(genescore)
# colnames(Z0) <- colnames(genescore)

In [8]:
gene.overlap <- intersect(rownames(X), rownames(Z0))
cell.overlap <- intersect(colnames(Y), colnames(Z0))

X <- as.matrix(X[gene.overlap,])
Z0 <- as.matrix(Z0[gene.overlap, cell.overlap])
Y <- as.matrix(Y[,cell.overlap])

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 3.1 GiB”


In [9]:
out <- dimReduce( dt1 =  X, dt2 = Z0,  K = 30)
x <- out$dt1
z0 <- out$dt2
y  <- dimReduce(dt1 = Y, K=30)

In [10]:
res <- BiCCA( X = t(x) ,
             Y = t(y), 
             Z0 =t(z0), 
             X.clst = as.vector(rna$seurat_clusters),
             Y.clst = as.vector(peak$leiden),
             alpha = 0.1, 
             lambda = 0.7,
             K = 15,
             temp.path  = "out",
             num.iteration = 50,
             tolerance = 0.01,
             save = TRUE,
             parameter.optimize = FALSE,
             block.size = 0)

2025-01-13 15:05:59.104915  Started!



2025-01-13 15:05:59.120564  Dimension Check: X[30x2187] Y[30x2187] Z0[30x2187]

2025-01-13 15:05:59.120564  Decomposing started!

Warning message in irlba(in2, nv = K, nu = K):
“You're computing too large a percentage of total singular values, use a standard svd instead.”
Warning message in irlba(in2, nv = K, nu = K):
“You're computing too large a percentage of total singular values, use a standard svd instead.”
Warning message in irlba(in2, nv = K, nu = K):
“You're computing too large a percentage of total singular values, use a standard svd instead.”
Warning message in irlba(in2, nv = K, nu = K):
“You're computing too large a percentage of total singular values, use a standard svd instead.”
Warning message in irlba(in2, nv = K, nu = K):
“You're computing too large a percentage of total singular values, use a standard svd instead.”
Warning message in irlba(in2, nv = K, nu = K):
“You're computing too large a percentage of total singular values, use a standard svd instead.”
Warning mess

In [11]:
library(umap)
dim(res$u)
dim(res$r)
embed = rbind(res$u, res$r)

[1] 2187   15

[1] 2187   15

In [12]:
# write.csv(file="rep5/BindSC_embed.csv",embed)